In [1]:
from bs4 import BeautifulSoup
import random
# Requests
import requests
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [2]:
franquicias_hist = pd.read_csv('../datos_finales/franquicias_historicas.csv')
standings_2003 = pd.read_csv('../datos_finales/standings_2003.csv')
salarios_jordan = pd.read_csv('../datos_finales/salarios_jordan.csv')
df_regular_season_jordan = pd.read_csv('../datos_finales/regular_season_jordan.csv')
df_temporadas_jordan = pd.read_csv('../datos_finales/temporadas_jordan.csv')
df_playoffs_jordan = pd.read_csv('../datos_finales/playoffs_jordan.csv')
df_campeonato_jordan = pd.read_csv('../datos_finales/campeonatos_jordan.csv')
standings_2024 = pd.read_csv('../datos_finales/standings_2024.csv')
salarios_lebron = pd.read_csv('../datos_finales/salarios_lebron.csv')
df_regular_season_lebron = pd.read_csv('../datos_finales/regular_season_lebron.csv')
df_playoff_lebron = pd.read_csv('../datos_finales/playoffs_lebron.csv')
df_temporadas_lebron = pd.read_csv('../datos_finales/temporadas_lebron.csv')
df_campeonato_lebron = pd.read_csv('../datos_finales/campeonatos_lebron.csv')

In [3]:
try:
    conexion = psycopg2.connect(
    database = 'Supermercados',
    user = 'postgres',
    password = 'admin',
    host = 'localhost',
    port = '5432')
except OperationalError as e: #el alias es e
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print('la contraseña es erronea')
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print('error de conexion')
    else:
        print(f'el error es el siguiente {e}')

In [4]:
standings_2024.head()

,Equipo,Nombre_comun,Codigo_equipo,Posicion,Partidos,Victorias,Derrotas,Porcentaje_vic
0,Boston Celtics,Celtics,BOS,1,82,64,18,0.780
1,Denver Nuggets,Nuggets,DEN,2,82,57,25,0.695
2,Oklahoma City Thunder,Thunder,OKC,3,82,57,25,0.695
3,Minnesota Timberwolves,Timberwolves,MIN,4,82,56,26,0.683
4,Los Angeles Clippers,Clippers,LAC,5,82,51,31,0.622


In [5]:
standings_2024.dtypes

Equipo             object
Nombre_comun       object
Codigo_equipo      object
Posicion            int64
Partidos            int64
Victorias           int64
Derrotas            int64
Porcentaje_vic    float64
dtype: object

In [6]:
salarios_jordan['Temporada'] = salarios_jordan['Temporada'].str.replace('/', '-')

In [7]:
# Realizar el merge en las columnas de temporada
salarios_jordan = salarios_jordan.merge(df_temporadas_jordan[['Id_temporada', 'SEASON_ID']], 
                                  left_on='Temporada', 
                                  right_on='SEASON_ID', 
                                  how='left')

# Eliminar la columna duplicada 'SEASON_ID' después del merge
salarios_jordan.drop(columns=['Temporada'], inplace=True)

# Mostrar el resultado
salarios_jordan.head()


,Equipo,team_code,Salario,Salario Actualizado,Id_temporada,SEASON_ID
0,Washington Wizards,WAS,1030000,1746871,15,2002-03
1,Washington Wizards,WAS,1000000,1714095,14,2001-02
2,Chicago Bulls,CHI,33140000,63077431,13,1997-98
3,Chicago Bulls,CHI,30140000,58685292,12,1996-97
4,Chicago Bulls,CHI,3850000,7702751,11,1995-96


In [8]:
salarios_jordan.drop(columns=['Equipo'])

,team_code,Salario,Salario Actualizado,Id_temporada,SEASON_ID
0,WAS,1030000,1746871,15,2002-03
1,WAS,1000000,1714095,14,2001-02
2,CHI,33140000,63077431,13,1997-98
3,CHI,30140000,58685292,12,1996-97
4,CHI,3850000,7702751,11,1995-96
5,CHI,3850000,7936957,10,1994-95
6,CHI,4000000,8704964,9,1992-93
7,CHI,3250000,7291207,8,1991-92
8,CHI,2500000,5871997,7,1990-91


In [9]:
salarios_lebron['Temporada'] = salarios_lebron['Temporada'].str.replace('/', '-')

In [10]:
salarios_lebron.head()

,Temporada,Equipo,team_code,Salario,Salario Actualizado
0,2023-24,Los Angeles Lakers,LAL,47607350,47607350
1,2022-23,Los Angeles Lakers,LAL,44474988,45795529
2,2021-22,Los Angeles Lakers,LAL,41180544,46244900
3,2020-21,Los Angeles Lakers,LAL,39219566,46417307
4,2019-20,Los Angeles Lakers,LAL,37436858,44593536


In [11]:
df_temporadas_lebron.head()

,Id_temporada,SEASON,TEAM_ABBREVIATION
0,1,2003-04,CLE
1,2,2004-05,CLE
2,3,2005-06,CLE
3,4,2006-07,CLE
4,5,2007-08,CLE


In [12]:
salarios_lebron = salarios_lebron.merge(df_temporadas_lebron[['Id_temporada', 'SEASON']], 
                                  left_on='Temporada', 
                                  right_on='SEASON', 
                                  how='left')

# Eliminar la columna duplicada 'SEASON' después del merge
salarios_lebron.drop(columns=['Temporada'], inplace=True)

# Mostrar el resultado
salarios_lebron.head()

,Equipo,team_code,Salario,Salario Actualizado,Id_temporada,SEASON
0,Los Angeles Lakers,LAL,47607350,47607350,21,2023-24
1,Los Angeles Lakers,LAL,44474988,45795529,20,2022-23
2,Los Angeles Lakers,LAL,41180544,46244900,19,2021-22
3,Los Angeles Lakers,LAL,39219566,46417307,18,2020-21
4,Los Angeles Lakers,LAL,37436858,44593536,17,2019-20


In [13]:
salarios_lebron

,Equipo,team_code,Salario,Salario Actualizado,Id_temporada,SEASON
0,Los Angeles Lakers,LAL,47607350,47607350,21,2023-24
1,Los Angeles Lakers,LAL,44474988,45795529,20,2022-23
2,Los Angeles Lakers,LAL,41180544,46244900,19,2021-22
3,Los Angeles Lakers,LAL,39219566,46417307,18,2020-21
4,Los Angeles Lakers,LAL,37436858,44593536,17,2019-20
5,Los Angeles Lakers,LAL,35654150,43170146,16,2018-19
6,Cleveland Cavaliers,CLE,33285709,41459735,15,2017-18
7,Cleveland Cavaliers,CLE,30963450,39197185,14,2016-17
8,Cleveland Cavaliers,CLE,22970500,29368777,13,2015-16
9,Cleveland Cavaliers,CLE,20644400,26427427,12,2014-15
